In [33]:
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

# add the path to my packages to system paths so they can be imported
import sys
sys.path.append('/home/yasamanparhizkar/Documents/yorku/01_thesis/code/my_packages')
# sys.path.append('F:\MAScThesis\code\my_packages')
# sys.path.append('/home/yasamanparhizkar/Documents/thesis/code/my_packages')

import data_handler_01 as dh
warnings.filterwarnings("ignore")

In [4]:
diamonds = sns.load_dataset("diamonds")
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [5]:
type(diamonds)

pandas.core.frame.DataFrame

In [6]:
print(diamonds)

       carat        cut color clarity  depth  table  price     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...    ...   ...   ...   ...
53935   0.72      Ideal     D     SI1   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72       Good     D     SI1   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70  Very Good     D     SI1   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    Premium     H     SI2   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 columns]


# Create a pandas dataframe from fish movie features

In [8]:
# load all spike data from file
spikes_dp = '../../data/original_files/spikes.csv'
binned_data = np.loadtxt(spikes_dp, delimiter=',')
binned_data = binned_data.reshape(binned_data.shape[0], 1141, 113)
binned_data = binned_data * 2 - 1     # turn labels from 0,1 to -1,1

I_order_10 = [54, 35, 10, 60, 74, 9, 61, 56, 91, 104]

# group all neurons together
grouped_data = np.zeros((297, 1141, 1))
for trial in range(297):
    for frame in range(1141):
        grouped_data[trial, frame, :] = 2 * int((binned_data[trial, frame, :] == 1).any()) - 1
        
print('grouped_data.shape = ', grouped_data.shape)

grouped_data.shape =  (297, 1141, 1)


In [66]:
# use only the second trial
ind_min = 1*1141+0
ind_max = 2*1141-1

def transform(fv):
    """
    Transform to be applied on feature vectors.
    
    Input: fv
    fv - 1xDf torch tensor representing a feature vector
    
    Output: fvv
    fvv - 1xDf' torch tensor representing the transformed feature vector
    """
    
    # for faster run and less memory usage
    fvv = fv[:, ::10]
    
    # for numerical stability during GD
    # fvv = fvv * 10
    
    return fvv

data_params = {'func': dh.datapoint_torch, 'features_dp': '../../data/features/slowfast/slowfast_4732/', \
               'spike_data': grouped_data, 'group_id': 0, 'transform': transform}

train_num = 500
val_num = 100
seed = None

# create training and validation datasets
train_num, val_num, train_data, val_data = dh.random_train_val(train_num, val_num, ind_min, ind_max, data_params, seed)

print('# training datapoints: ', train_num)
print('# validation datapoints: ', val_num)
print('features array shape (NxDf): ', train_data['des'].shape)
print('a few rows of the training feature array:\n',train_data['des'][:5])

# training datapoints:  500
# validation datapoints:  100
features array shape (NxDf):  (500, 474)
a few rows of the training feature array:
 [[2.7171607 1.8891053 1.8667963 ... 2.6452    5.0123525 2.1651871]
 [2.7342947 1.7744384 1.9787449 ... 2.740112  5.0628047 1.9402164]
 [2.7427316 1.9135818 1.8079212 ... 2.7134817 4.9467864 1.6767261]
 [2.7964344 1.8926489 1.9096606 ... 2.6237352 4.9819055 1.6886238]
 [2.8321726 1.881557  1.8612114 ... 2.677793  4.988236  1.7620839]]


In [67]:
# create a dataframe from the features array
train_df = pd.DataFrame(data = train_data['des'])
val_df = pd.DataFrame(data = val_data['des'])
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,464,465,466,467,468,469,470,471,472,473
0,2.717161,1.889105,1.866796,3.155151,2.786354,7.330894,4.032849,5.465666,5.330612,3.410649,...,2.435522,2.705825,2.344805,4.701004,2.213653,3.146791,1.866297,2.645200,5.012352,2.165187
1,2.734295,1.774438,1.978745,3.171515,2.795883,7.376439,4.199847,5.498415,5.542151,4.326243,...,2.250646,2.709613,2.124864,4.607258,2.210824,2.178204,3.041438,2.740112,5.062805,1.940216
2,2.742732,1.913582,1.807921,3.225475,2.826887,7.457115,4.174360,5.358123,5.497444,3.035140,...,2.103599,2.698660,2.131648,4.696541,2.199164,2.350266,1.931064,2.713482,4.946786,1.676726
3,2.796434,1.892649,1.909661,3.225241,2.790021,7.357202,4.002663,5.545738,5.499510,3.031996,...,2.296342,2.678228,2.167713,4.647845,2.220382,1.688968,1.981825,2.623735,4.981905,1.688624
4,2.832173,1.881557,1.861211,3.045776,2.624912,3.611086,4.160274,5.302925,5.464644,3.126502,...,2.274695,2.719636,2.056880,4.718672,2.214847,2.317703,1.914328,2.677793,4.988236,1.762084


In [68]:
# create pandas series from the labels
train_y = pd.DataFrame(data = train_data['lbls'])
val_y = pd.DataFrame(data = val_data['lbls'])
train_y.head()

,0
0,-1.0
1,1.0
2,1.0
3,1.0
4,-1.0


In [69]:
# create xgb Dmatrices from dataframes
dtrain_from_df = xgb.DMatrix(train_df, train_y)
print('Dmatrix shape: {} x {}'.format(dtrain_from_df.num_row(), dtrain_from_df.num_col()))
print('Dmatrix labels: ', dtrain_from_df.get_label())

Dmatrix shape: 500 x 474
Dmatrix labels:  [-1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1. -1.
  1.  1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1.  1.  1.  1. -1.  1.  1.
  1.  1.  1.  1. -1.  1. -1. -1.  1.  1.  1.  1. -1.  1.  1. -1.  1. -1.
 -1. -1. -1. -1.  1.  1. -1.  1. -1.  1.  1.  1.  1.  1. -1. -1.  1.  1.
  1. -1.  1. -1.  1.  1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.
 -1.  1.  1. -1. -1.  1.  1.  1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.
 -1.  1.  1.  1.  1.  1. -1. -1.  1. -1.  1.  1.  1. -1.  1. -1.  1.  1.
  1.  1. -1.  1.  1. -1.  1.  1. -1.  1. -1. -1. -1. -1. -1.  1.  1. -1.
  1.  1.  1. -1.  1. -1. -1.  1.  1.  1.  1.  1. -1.  1.  1. -1. -1. -1.
 -1.  1.  1.  1. -1. -1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.
  1. -1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.  1. -1.
 -1.  1. -1. -1.  1.  1.  1. -1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1.
  1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.
  1.  1. 

In [70]:
# create xgb Dmatrices directly from numpy arrays
dtrain = xgb.DMatrix(train_data['des'], label=(train_data['lbls']+1)//2)
dval = xgb.DMatrix(val_data['des'], label=(val_data['lbls']+1)//2)

In [71]:
print('Train Dmatrix shape: {} x {}'.format(dtrain.num_row(), dtrain.num_col()))
print('Val Dmatrix shape: {} x {}'.format(dval.num_row(), dval.num_col()))
print('Train Dmatrix labels: ', dtrain.get_label())
print('Val Dmatrix labels: ', dval.get_label())

Train Dmatrix shape: 500 x 474
Val Dmatrix shape: 100 x 474
Train Dmatrix labels:  [0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1.
 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1.
 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1.
 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1.
 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1

# Training

In [75]:
# Define hyperparameters
params = {"objective": "binary:logistic", "tree_method": "hist"}
evals = [(dval, "validation"), (dtrain, "train")]
n = 20
veval = 1

model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
    evals=evals,
    verbose_eval=veval # print losses every ten rounds
)

[0]	validation-logloss:0.59869	train-logloss:0.51123
[1]	validation-logloss:0.55499	train-logloss:0.39508
[2]	validation-logloss:0.53096	train-logloss:0.32590
[3]	validation-logloss:0.53427	train-logloss:0.26129
[4]	validation-logloss:0.51069	train-logloss:0.21263
[5]	validation-logloss:0.49596	train-logloss:0.17372
[6]	validation-logloss:0.50618	train-logloss:0.14679
[7]	validation-logloss:0.50679	train-logloss:0.12822
[8]	validation-logloss:0.50771	train-logloss:0.10872
[9]	validation-logloss:0.50709	train-logloss:0.09398
[10]	validation-logloss:0.50608	train-logloss:0.08202
[11]	validation-logloss:0.51360	train-logloss:0.07275
[12]	validation-logloss:0.51259	train-logloss:0.06459
[13]	validation-logloss:0.51520	train-logloss:0.05759
[14]	validation-logloss:0.51328	train-logloss:0.05217
[15]	validation-logloss:0.52012	train-logloss:0.04688
[16]	validation-logloss:0.50957	train-logloss:0.04244
[17]	validation-logloss:0.52250	train-logloss:0.03896
[18]	validation-logloss:0.53158	train-

In [81]:
model2 = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
    evals=evals,
    verbose_eval=veval, # print losses every ten rounds
    early_stopping_rounds=1 # activate early stopping
)

[0]	validation-logloss:0.59869	train-logloss:0.51123
[1]	validation-logloss:0.55499	train-logloss:0.39508
[2]	validation-logloss:0.53096	train-logloss:0.32590
[3]	validation-logloss:0.53427	train-logloss:0.26129
[4]	validation-logloss:0.51069	train-logloss:0.21263
[5]	validation-logloss:0.49596	train-logloss:0.17372
[6]	validation-logloss:0.50618	train-logloss:0.14679
[7]	validation-logloss:0.50679	train-logloss:0.12822
[8]	validation-logloss:0.50771	train-logloss:0.10872
[9]	validation-logloss:0.50709	train-logloss:0.09398
[10]	validation-logloss:0.50608	train-logloss:0.08202
[11]	validation-logloss:0.51360	train-logloss:0.07275
[12]	validation-logloss:0.51259	train-logloss:0.06459
[13]	validation-logloss:0.51520	train-logloss:0.05759
[14]	validation-logloss:0.51328	train-logloss:0.05217
[15]	validation-logloss:0.52012	train-logloss:0.04688
[16]	validation-logloss:0.50957	train-logloss:0.04244
[17]	validation-logloss:0.52250	train-logloss:0.03896
[18]	validation-logloss:0.53158	train-

In [85]:
# cross-validation
results = xgb.cv(
    params=params, 
    dtrain=dtrain,
    num_boost_round=n,
    nfold=5,
    metrics=["logloss", "auc", "error"]
)

In [87]:
results.head()

,train-logloss-mean,train-logloss-std,train-auc-mean,train-auc-std,train-error-mean,train-error-std,test-logloss-mean,test-logloss-std,test-auc-mean,test-auc-std,test-error-mean,test-error-std
0,0.508132,0.006270,0.972983,0.008762,0.0795,0.005788,0.616481,0.026422,0.731452,0.058901,0.314,0.057131
1,0.391002,0.005626,0.994763,0.000982,0.0365,0.012309,0.559781,0.043146,0.765420,0.059205,0.276,0.066212
2,0.309110,0.006440,0.998074,0.000782,0.0170,0.003674,0.526749,0.059508,0.791202,0.062296,0.252,0.057758
3,0.245860,0.004935,0.999762,0.000257,0.0055,0.003317,0.500056,0.064319,0.811616,0.058010,0.218,0.066453
4,0.199654,0.005781,0.999905,0.000117,0.0020,0.001000,0.487882,0.072749,0.817722,0.060889,0.226,0.067112


In [89]:
# hyperparameter tuning with cross-validation

# Validation

In [90]:
preds = model.predict(dval)

In [93]:
print('preds tyep: ', type(preds))
print('preds shape: ', preds.shape)
print('preds:\n',preds)

preds tyep:  <class 'numpy.ndarray'>
preds shape:  (100,)
preds:
 [0.33795464 0.9734629  0.12072904 0.9960601  0.08313764 0.46035868
 0.7917764  0.96411616 0.7684967  0.89821446 0.89954054 0.35246402
 0.9395617  0.89338136 0.2709972  0.9884241  0.991553   0.1372957
 0.9111583  0.22615255 0.59044766 0.886293   0.6430878  0.99099535
 0.21737342 0.92895836 0.07304604 0.98452675 0.29367244 0.13176289
 0.9742159  0.92201746 0.9881207  0.965901   0.9710659  0.963851
 0.9866866  0.3645265  0.04148287 0.59836787 0.951548   0.99273914
 0.75967115 0.98820335 0.98685116 0.88409835 0.7781451  0.63237274
 0.9563478  0.95762193 0.97649515 0.9344622  0.97822624 0.33608955
 0.9536734  0.98340076 0.7211842  0.7145676  0.9114022  0.99242955
 0.5645949  0.75682944 0.98043984 0.9810286  0.61620265 0.98410565
 0.9153689  0.98762727 0.99475265 0.9866536  0.46690845 0.8443176
 0.6204501  0.99104136 0.93520904 0.93287086 0.38465348 0.93032485
 0.99072343 0.9554291  0.9946272  0.9472902  0.89231074 0.9679501
 

In [102]:
preds_th = (preds > 0.5).astype(int)
gt_lbls = (val_data['lbls']+1)//2
acc = sum(preds_th == gt_lbls)/len(gt_lbls)
err = sum(preds_th != gt_lbls)/len(gt_lbls)


print('Thresholded preds:\n', preds_th)
print('Ground-truth val. labels:\n', gt_lbls)
print('Val. accuracy: ', acc)
print('Val. error rate: ', err)

Thresholded preds:
 [0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1 1 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1]
Ground-truth val. labels:
 [1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1.
 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0.
 1. 1. 1. 1.]
Val. accuracy:  0.76
Val. error rate:  0.24
